## Analyzing A/B Test Results



## Table of Contents
- [Introduction](#intro)
- [Part I - Probability](#probability)
- [Part II - A/B Test](#ab_test)
- [Part III - Regression](#regression)


<a id='intro'></a>
### Introduction



<a id='probability'></a>
#### Part I - Probability

To get started, let's import our libraries.

In [ ]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline
#We are setting the seed to assure you get the same answers on quizzes as we set up
random.seed(42)

`1.` Now, read in the `ab_data.csv` data. Store it in `df`.  **Use your dataframe to answer the questions in Quiz 1 of the classroom.**

a. Read in the dataset and take a look at the top few rows here:

In [ ]:
df=pd.read_csv('/kaggle/input/ab-data/ab_data.csv')
df.head()

b. Use the cell below to find the number of rows in the dataset.

In [ ]:
df.shape[0]

c. The number of unique users in the dataset.

In [ ]:
df.user_id.nunique()

d. The proportion of users converted.

In [ ]:
df.converted.mean()

e. The number of times the `new_page` and `treatment` don't match.

In [ ]:
(df.groupby('landing_page')).group.value_counts()

In [ ]:
1928+1965

f. Do any of the rows have missing values?

In [ ]:
df.isnull().sum()

`2.` For the rows where **treatment** does not match with **new_page** or **control** does not match with **old_page**, we cannot be sure if this row truly received the new or old page.  Use **Quiz 2** in the classroom to figure out how we should handle these rows.  

a. Now use the answer to the quiz to create a new dataset that meets the specifications from the quiz.  Store your new dataframe in **df2**.

In [ ]:
df2=df.copy()
dropped = df2.query('(landing_page=="new_page" & group == "control") or (landing_page=="old_page" & group == "treatment")')
df2.drop(dropped.index, axis=0, inplace=True)

In [ ]:
# Double Check all of the correct rows were removed - this should be 0
df2[((df2['group'] == 'treatment') == (df2['landing_page'] == 'new_page')) == False].shape[0]

`3.` Use **df2** and the cells below to answer questions for **Quiz3** in the classroom.

a. How many unique **user_id**s are in **df2**?

In [ ]:
df2.user_id.nunique()

b. There is one **user_id** repeated in **df2**.  What is it?

In [ ]:
df2[df2['user_id'].duplicated()] 

c. What is the row information for the repeat **user_id**? 

In [ ]:
df2.query('user_id==773192')

d. Remove **one** of the rows with a duplicate **user_id**, but keep your dataframe as **df2**.

In [ ]:
df2.drop(2893,axis=0,inplace=True)

`4.` Use **df2** in the cells below to answer the quiz questions related to **Quiz 4** in the classroom.

a. What is the probability of an individual converting regardless of the page they receive?

In [ ]:
df2.converted.mean()

b. Given that an individual was in the `control` group, what is the probability they converted?

In [ ]:
control_conv = df2.query('group=="control"').converted.mean()
control_conv

c. Given that an individual was in the `treatment` group, what is the probability they converted?

In [ ]:
ttt_conv = df2.query('group=="treatment"').converted.mean()
ttt_conv

d. What is the probability that an individual received the new page?

In [ ]:
df2.query('landing_page=="new_page"').shape[0]/df2.shape[0]

e. Consider your results from parts (a) through (d) above, and explain below whether you think there is sufficient evidence to conclude that the new treatment page leads to more conversions.

> There is no evidence that the new page leads to more conversions as the mean conversions for the treatment(new page) group is less than the mean conversions for the control(old page) group.

<a id='ab_test'></a>
### Part II - A/B Test

Notice that because of the time stamp associated with each event, you could technically run a hypothesis test continuously as each observation was observed.  

However, then the hard question is do you stop as soon as one page is considered significantly better than another or does it need to happen consistently for a certain amount of time?  How long do you run to render a decision that neither page is better than another?  

These questions are the difficult parts associated with A/B tests in general.  


`1.` For now, consider you need to make the decision just based on all the data provided.  If you want to assume that the old page is better unless the new page proves to be definitely better at a Type I error rate of 5%, what should your null and alternative hypotheses be?  You can state your hypothesis in terms of words or in terms of **$p_{old}$** and **$p_{new}$**, which are the converted rates for the old and new pages.

> Null : P<sub>new</sub> - P<sub>old</sub> <= 0

> Alternative : P<sub>new</sub> - P<sub>old</sub> > 0                                 

`2.` Assume under the null hypothesis, $p_{new}$ and $p_{old}$ both have "true" success rates equal to the **converted** success rate regardless of page - that is $p_{new}$ and $p_{old}$ are equal. Furthermore, assume they are equal to the **converted** rate in **ab_data.csv** regardless of the page. <br><br>

Use a sample size for each page equal to the ones in **ab_data.csv**.  <br><br>

Perform the sampling distribution for the difference in **converted** between the two pages over 10,000 iterations of calculating an estimate from the null.  <br><br>

Use the cells below to provide the necessary parts of this simulation.  If this doesn't make complete sense right now, don't worry - you are going to work through the problems below to complete this problem.  You can use **Quiz 5** in the classroom to make sure you are on the right track.<br><br>

a. What is the **conversion rate** for $p_{new}$ under the null? 

In [ ]:
p_new = df2.converted.mean()
p_new

b. What is the **conversion rate** for $p_{old}$ under the null? <br><br>

In [ ]:
p_old = df2.converted.mean()
p_old

c. What is $n_{new}$, the number of individuals in the treatment group?

In [ ]:
n_new = df2.query('group=="treatment"').shape[0]
n_new

d. What is $n_{old}$, the number of individuals in the control group?

In [ ]:
n_old = df2.query('group=="control"').shape[0]
n_old

e. Simulate $n_{new}$ transactions with a conversion rate of $p_{new}$ under the null.  Store these $n_{new}$ 1's and 0's in **new_page_converted**.

In [ ]:
new_page_converted = np.random.binomial(1,p=p_new,size=n_new)

f. Simulate $n_{old}$ transactions with a conversion rate of $p_{old}$ under the null.  Store these $n_{old}$ 1's and 0's in **old_page_converted**.

In [ ]:
old_page_converted = np.random.binomial(1,p=p_old,size=n_old)

g. Find $p_{new}$ - $p_{old}$ for your simulated values from part (e) and (f).

In [ ]:
obs_diff = new_page_converted.mean()-old_page_converted.mean()
obs_diff

h. Create 10,000 $p_{new}$ - $p_{old}$ values using the same simulation process you used in parts (a) through (g) above. Store all 10,000 values in a NumPy array called **p_diffs**.

In [ ]:
new_page_converted = np.random.binomial(n_new, p_new, 10000)/n_new
old_page_converted = np.random.binomial(n_old, p_old, 10000)/n_old
p_diffs = new_page_converted - old_page_converted

i. Plot a histogram of the **p_diffs**.  Does this plot look like what you expected?  Use the matching problem in the classroom to assure you fully understand what was computed here.

In [ ]:
ab_diff = df2.query('landing_page=="new_page"').converted.mean()-df2.query('landing_page=="old_page"').converted.mean()

plt.hist(p_diffs);
plt.axvline(x=ab_diff,color='red',linewidth=5);
plt.xlabel('Difference in conversion rates');
plt.ylabel('Counts of the difference in conversion rates');
plt.title('Difference in conversion rates between the two pages under the null')
plt.show();

j. What proportion of the **p_diffs** are greater than the actual difference observed in **ab_data.csv**?

In [ ]:
(p_diffs>ab_diff).mean()

k. Please explain using the vocabulary you've learned in this course what you just computed in part **j.**  What is this value called in scientific studies?  What does this value mean in terms of whether or not there is a difference between the new and old pages?

> the p value: It means that the probability of achieveing (our observed difference or a more extreme difference in favor of the alternative hypothesis) due to pure chance (while the null hypothesis is true with no real difference in the conversion rates between the two pages).

l. We could also use a built-in to achieve similar results.  Though using the built-in might be easier to code, the above portions are a walkthrough of the ideas that are critical to correctly thinking about statistical significance. Fill in the below to calculate the number of conversions for each page, as well as the number of individuals who received each page. Let `n_old` and `n_new` refer the the number of rows associated with the old page and new pages, respectively.

In [ ]:
import statsmodels.api as sm

convert_old = df2.query('landing_page=="old_page"').converted.sum()
convert_new = df2.query('landing_page=="new_page"').converted.sum()
n_old = df2.query('landing_page=="old_page"').shape[0]
n_new = df2.query('landing_page=="new_page"').shape[0]

m. Now use `stats.proportions_ztest` to compute your test statistic and p-value.  [Here](https://docs.w3cub.com/statsmodels/generated/statsmodels.stats.proportion.proportions_ztest/) is a helpful link on using the built in.

In [ ]:
import statsmodels.stats as stats
stats.proportion.proportions_ztest([convert_new,convert_old],[n_new,n_old],alternative='larger')

n. What do the z-score and p-value you computed in the previous question mean for the conversion rates of the old and new pages?  Do they agree with the findings in parts **j.** and **k.**?

> The z-score describes the distance from the mean on the standard normal distribution. The z-score is less than the z-critical value of 1.96, so we are likely to have such a sample from the sampling distribution given an alpha of 0.05.

> The p-value means the 90% of the conversion rates under the null hypothesis are in favor of the alternative hypothesis more than our observed difference and this is based on pure chance not on a real difference in the conversion rates between the new and the old pages. There is no sufficient evidence to reject the null hypothesis.

> Yes, the findings agree with the findings in parts j and k.

<a id='regression'></a>
### Part III - A regression approach

`1.` In this final part, you will see that the result you achieved in the A/B test in Part II above can also be achieved by performing regression.<br><br> 

a. Since each row is either a conversion or no conversion, what type of regression should you be performing in this case?

> Logsitic Regression

b. The goal is to use **statsmodels** to fit the regression model you specified in part **a.** to see if there is a significant difference in conversion based on which page a customer receives. However, you first need to create in df2 a column for the intercept, and create a dummy variable column for which page each user received.  Add an **intercept** column, as well as an **ab_page** column, which is 1 when an individual receives the **treatment** and 0 if **control**.

In [ ]:
df2['intercept']=1
df2['ab_page']=pd.get_dummies(df['group'])['treatment']

c. Use **statsmodels** to instantiate your regression model on the two columns you created in part b., then fit the model using the two columns you created in part **b.** to predict whether or not an individual converts. 

In [ ]:
lm=sm.Logit(df2['converted'],df2[['intercept','ab_page']])
results=lm.fit()

d. Provide the summary of your model below, and use it as necessary to answer the following questions.

In [ ]:
results.summary2()

e. What is the p-value associated with **ab_page**? Why does it differ from the value you found in **Part II**?<br><br>

> The p value of ab_page is 0.19

> The p-value with the ab_pagee is the probability that we obtain (this difference in conversion rate or more extreme difference between the old and new page) due to pure chance. So, this is a two-sided test where the alternative hypothesis is that the difference between the 2 pages doesn't equal 0. On the contrary, the alternative hypothesis in part 2 was that the new page is associated with higher conversion rate than the old page, so it's a one-sided test.

> In the logistic regression model, the p value is the probability distribution from the 2 tails on the right and left of the + or - z score, while in part 2 the p value is the probability distribution right to the z score which is negative meaning that the p value will be greater than 50%

f. Now, you are considering other things that might influence whether or not an individual converts.  Discuss why it is a good idea to consider other factors to add into your regression model.  Are there any disadvantages to adding additional terms into your regression model?

> We should consider other predictive factors so that we could predict the conversion rate better and fit a better model. However, adding additional terms that has no independent correlation with the conversion rate will mislead us when they are correlated to other predictive variables (multicolinearity). This will cause coefficients to be flipped away from what is predicted from the simple linear regression model and bivariate plots.

g. Now along with testing if the conversion rate changes for different pages, also add an effect based on which country a user lives in. You will need to read in the **countries.csv** dataset and merge together your datasets on the appropriate rows.  [Here](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.join.html) are the docs for joining tables. 

Does it appear that country had an impact on conversion?  Don't forget to create dummy variables for these country columns - **Hint: You will need two columns for the three dummy variables.** Provide the statistical output as well as a written response to answer this question.

In [ ]:
df_countries = pd.read_csv('/kaggle/input/countries/countries.csv')
df_countries.head()

In [ ]:
df2 = df2.set_index('user_id').join(df_countries.set_index('user_id'))
df2.head()

In [ ]:
df2.country.unique()

In [ ]:
df2[['CA','UK','US']]=pd.get_dummies(df2['country'])
df2.drop('US',axis=1,inplace=True)

In [ ]:
lm=sm.Logit(df2['converted'],df2[['intercept','ab_page','CA','UK']])
results=lm.fit()
results.summary2()

> The country impact on the conversion rate isn't statistically significant as it has a p-value greater than 0.05

h. Though you have now looked at the individual factors of country and page on conversion, we would now like to look at an interaction between page and country to see if there significant effects on conversion.  Create the necessary additional columns, and fit the new model.  

Provide the summary results, and your conclusions based on the results.

In [ ]:
df2['page_CA'] = df2['ab_page']*df2['CA']
df2['page_UK']=df2['ab_page']*df2['UK']
lm=sm.Logit(df2['converted'],df2[['intercept','ab_page','CA','UK','page_CA','page_UK']])
results=lm.fit()
results.summary2()

> The coefficients associated with the interaction between the page and the countries aren't statistically significant and there's an overlap between their confidence intervals, So we don't have enough evidence to conclude that there's significant interaction between country and page.

## Summary & Conclusion

> Our p values aren't statistically significant despite multiple testing, so we don't have enough evidence to say that the new page or countries have an impact on the conversion rate.

> I think our sample size is large enough. Although running the experiment for larger time might give us statistically significant p-values, the effect is unlikely to have practical significance.

> Based on our results, we conclude that the company should keep the old page.